In [8]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [9]:
img_transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print(x)
else:
    print("MPS device not found.")

tensor([1.], device='mps:0')


In [10]:
for x, y in dataloader:
    print(x.shape)
    break

torch.Size([2, 1, 28, 28])


In [14]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=1, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=1, kernel_size=2, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5, stride=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=3)
        
        self.conv4 = nn.Conv2d(in_channels=15, out_channels=10, kernel_size=2, stride=3)
        self.conv5 = nn.Conv2d(in_channels=10, out_channels=5, kernel_size=3, stride=3)
        self.conv6 = nn.Conv2d(in_channels=5, out_channels=1, kernel_size=5, stride=3)
    
    def encoder(self, x):
        h = self.pool1(torch.relu(self.conv1(x)))
        h = self.pool2(torch.relu(self.conv2(h)))
        h = self.pool3(torch.relu(self.conv3(h)))
        return h
    
    def decoder(self, x):
        h = nn.functional.interpolate(x, scale_factor=10)
        h = torch.relu(self.conv4(h))
        h = nn.functional.interpolate(x, scale_factor=2)
        h = torch.relu(self.conv5(h))
        h = nn.functional.interpolate(h, scale_factor=2)
        h = torch.relu(self.conv6(h))
        return h
    
    def forward(self, x):
        h = self.encoder(x)
        xtilde = self.decoder(h)
        return xtilde

In [12]:
model = AE()

In [15]:
model(x)

RuntimeError: Given input size: (1x2x2). Calculated output size: (1x0x0). Output size is too small

In [ ]:
model.forward(x)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], grad_fn=<ReluBackward0>)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [ ]:
loss_function = nn.MSELoss()

In [ ]:
for epoch in range(10):
    for x, _ in dataloader:
        pred = model(x)
        loss = loss_function(pred, x)
        loss.backward()
        optimizer.step()
    print(loss)

tensor(0.0622, grad_fn=<MseLossBackward0>)
tensor(0.0999, grad_fn=<MseLossBackward0>)
tensor(0.1070, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 